In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn import preprocessing
from sklearn import datasets
from google.cloud import storage

In [2]:
INPUT_PATH = "gs://kkbox-data/50_pct_undersample/"
split = "test"

#X = pd.read_csv(INPUT_PATH + f"X_{split}.csv")
X = pd.read_csv(INPUT_PATH + "{}_members.csv".format(split))
X.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,O0FgWPiZQJ16Tg9vJvwkm0936GHk+etJX7Vjf9FE9Ac=,0,1,0,female,7,20140927
1,1NiK3Fw0qWo5Q1UTa3Ud/RKbyRslunMKEu2gVwnX+68=,0,13,23,female,3,20130208
2,ROzh2KDSl9eKaoh2Rgz49LMi6sy/tQIaPqefKiCriSE=,0,5,32,male,9,20080201
3,Q8HPlo+J8BCkGZTZPveMSiCEnUfRDxzspZ1jMynRZAo=,0,12,28,female,7,20151117
4,c4phpsQQZzzVIvZA35SwyqHEq2b/XF++BjXfSNiSZeo=,0,14,35,female,9,20071222


# Gender
We treat 'unknown' as a 3rd category of gender with the assumption being that the user's decision to not provide gender may be relevant to our predictions

In [3]:
X.loc[X["gender"] == "male","gender"] = "M"
X.loc[X["gender"] == "female","gender"] = "F"
X.loc[:,"gender"]=X["gender"].fillna("U")

enc_gender = preprocessing.OneHotEncoder(drop="first",categories="auto")
gender_onehot = enc_gender.fit_transform(X.gender.values.reshape(-1,1)).toarray()
# Drop old gender column and merge new onehot encoded columns
X=X.drop(["gender"],axis=1)
X["gender_male"] = gender_onehot[:,0]
X["gender_unknown"] = gender_onehot[:,1]
X.head()

,msno,is_churn,city,bd,registered_via,registration_init_time,gender_male,gender_unknown
0,O0FgWPiZQJ16Tg9vJvwkm0936GHk+etJX7Vjf9FE9Ac=,0,1,0,7,20140927,0.0,0.0
1,1NiK3Fw0qWo5Q1UTa3Ud/RKbyRslunMKEu2gVwnX+68=,0,13,23,3,20130208,0.0,0.0
2,ROzh2KDSl9eKaoh2Rgz49LMi6sy/tQIaPqefKiCriSE=,0,5,32,9,20080201,1.0,0.0
3,Q8HPlo+J8BCkGZTZPveMSiCEnUfRDxzspZ1jMynRZAo=,0,12,28,7,20151117,0.0,0.0
4,c4phpsQQZzzVIvZA35SwyqHEq2b/XF++BjXfSNiSZeo=,0,14,35,9,20071222,0.0,0.0


# Registration Date
We convert registration date into an integer containing the number of days since registration. Those values are then standardized.

In [4]:
reg_date = pd.to_datetime(X.registration_init_time, format="%Y%m%d")
# Assuming here that our training date for predicting February 2017 is Jan 1, 2017
train_date = pd.to_datetime("20170131", format="%Y%m%d")
reg_days = (train_date - reg_date).dt.days

reg_days_scaled = preprocessing.scale(reg_days)
X["reg_days_scaled"] = pd.Series(reg_days_scaled)
X=X.drop(["registration_init_time"], axis=1)
X.head()

,msno,is_churn,city,bd,registered_via,gender_male,gender_unknown,reg_days_scaled
0,O0FgWPiZQJ16Tg9vJvwkm0936GHk+etJX7Vjf9FE9Ac=,0,1,0,7,0.0,0.0,-0.715149
1,1NiK3Fw0qWo5Q1UTa3Ud/RKbyRslunMKEu2gVwnX+68=,0,13,23,3,0.0,0.0,-0.159621
2,ROzh2KDSl9eKaoh2Rgz49LMi6sy/tQIaPqefKiCriSE=,0,5,32,9,1.0,0.0,1.549838
3,Q8HPlo+J8BCkGZTZPveMSiCEnUfRDxzspZ1jMynRZAo=,0,12,28,7,0.0,0.0,-1.102900
4,c4phpsQQZzzVIvZA35SwyqHEq2b/XF++BjXfSNiSZeo=,0,14,35,9,0.0,0.0,1.588054


# Registration Method
We one-hot encode the registration methods

In [5]:
enc_registration = preprocessing.OneHotEncoder(drop="first",categories="auto")
registration_onehot = enc_registration.fit_transform(X.registered_via.values.reshape(-1,1)).toarray()
registration_df = pd.DataFrame(registration_onehot)
registration_df.columns = enc_registration.get_feature_names(["registered_via"])
X = pd.concat([X,registration_df], axis=1)
X = X.drop(["registered_via"],axis=1)
X.head()

,msno,is_churn,city,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9
0,O0FgWPiZQJ16Tg9vJvwkm0936GHk+etJX7Vjf9FE9Ac=,0,1,0,0.0,0.0,-0.715149,0.0,1.0,0.0
1,1NiK3Fw0qWo5Q1UTa3Ud/RKbyRslunMKEu2gVwnX+68=,0,13,23,0.0,0.0,-0.159621,0.0,0.0,0.0
2,ROzh2KDSl9eKaoh2Rgz49LMi6sy/tQIaPqefKiCriSE=,0,5,32,1.0,0.0,1.549838,0.0,0.0,1.0
3,Q8HPlo+J8BCkGZTZPveMSiCEnUfRDxzspZ1jMynRZAo=,0,12,28,0.0,0.0,-1.102900,0.0,1.0,0.0
4,c4phpsQQZzzVIvZA35SwyqHEq2b/XF++BjXfSNiSZeo=,0,14,35,0.0,0.0,1.588054,0.0,0.0,1.0


# City
We one-hot encode the city

In [6]:
enc_city = preprocessing.OneHotEncoder(drop="first",categories="auto")
city_onehot = enc_city.fit_transform(X.city.values.reshape(-1,1)).toarray()
city_df = pd.DataFrame(city_onehot)
city_df.columns = enc_city.get_feature_names(["city"])
X = pd.concat([X,city_df], axis=1)
X = X.drop(["city"],axis=1)
X.head()

,msno,is_churn,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,O0FgWPiZQJ16Tg9vJvwkm0936GHk+etJX7Vjf9FE9Ac=,0,0,0.0,0.0,-0.715149,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1NiK3Fw0qWo5Q1UTa3Ud/RKbyRslunMKEu2gVwnX+68=,0,23,0.0,0.0,-0.159621,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ROzh2KDSl9eKaoh2Rgz49LMi6sy/tQIaPqefKiCriSE=,0,32,1.0,0.0,1.549838,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Q8HPlo+J8BCkGZTZPveMSiCEnUfRDxzspZ1jMynRZAo=,0,28,0.0,0.0,-1.102900,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,c4phpsQQZzzVIvZA35SwyqHEq2b/XF++BjXfSNiSZeo=,0,35,0.0,0.0,1.588054,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Birthday

Scaling the BD values. Others have indicated that this feature is predictive (despite the strange, negative values)

In [7]:
X["bd"] = preprocessing.scale(X.bd)
X.head()

,msno,is_churn,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,...,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22
0,O0FgWPiZQJ16Tg9vJvwkm0936GHk+etJX7Vjf9FE9Ac=,0,-0.928474,0.0,0.0,-0.715149,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1NiK3Fw0qWo5Q1UTa3Ud/RKbyRslunMKEu2gVwnX+68=,0,0.345747,0.0,0.0,-0.159621,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ROzh2KDSl9eKaoh2Rgz49LMi6sy/tQIaPqefKiCriSE=,0,0.844355,1.0,0.0,1.549838,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Q8HPlo+J8BCkGZTZPveMSiCEnUfRDxzspZ1jMynRZAo=,0,0.622751,0.0,0.0,-1.102900,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,c4phpsQQZzzVIvZA35SwyqHEq2b/XF++BjXfSNiSZeo=,0,1.010558,0.0,0.0,1.588054,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X.to_csv(INPUT_PATH + "{}_members_transformed.csv".format(split),index=False)